# Data Preparation and Basic Text Analysis - Topic Modeling Pt. 1

### #1. Zotero API + PDF Text Miner
We need this so we don't have to bring down all of the pdfs in our library to our local machines, and we can always run our data prep on the most up-to-date corpus.

In [13]:
import fitz
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
import ftfy
import nltk.corpus
from nltk.collocations import *
from nltk.corpus import stopwords
import pandas as pd
import datetime
import glob
import requests
import urllib.request
import os
import re
import time
import math
import json

In [2]:
def zoteroCrawler(key, groupid):
    multiplier = 0
    zoterogroup = []
    # get number of items in group
    headers = {"Zotero-API-Version":"3",'Connection':'close', "Zotero-API-Key":key}
    checkurl = "https://api.zotero.org/groups/" +groupid
    rcheck = requests.get(checkurl, headers=headers)
    items = rcheck.json()['meta']['numItems']
    print(items)
    pages = math.ceil((items/100))
    while pages > 0:
        url = "https://api.zotero.org/groups/" +groupid + "/items" + "?limit=100&start=" + str(multiplier)
        print(url)
        r = requests.get(url, headers=headers)
        rj = r.json() #jsonified version of our Zotero group
        print(len(rj))
        for i in rj:
            zoterogroup.append(i)
        multiplier = multiplier + 100
        pages = pages - 1
    return zoterogroup

In [3]:
api_key = "N6yPwqH9VQFt8ZKBKCAFf8KV"
group_id = "2808857"

In [4]:
rj= zoteroCrawler(api_key, group_id)

1117
https://api.zotero.org/groups/2808857/items?limit=100&start=0
100
https://api.zotero.org/groups/2808857/items?limit=100&start=100
100
https://api.zotero.org/groups/2808857/items?limit=100&start=200
100
https://api.zotero.org/groups/2808857/items?limit=100&start=300
100
https://api.zotero.org/groups/2808857/items?limit=100&start=400
100
https://api.zotero.org/groups/2808857/items?limit=100&start=500
100
https://api.zotero.org/groups/2808857/items?limit=100&start=600
100
https://api.zotero.org/groups/2808857/items?limit=100&start=700
100
https://api.zotero.org/groups/2808857/items?limit=100&start=800
100
https://api.zotero.org/groups/2808857/items?limit=100&start=900
100
https://api.zotero.org/groups/2808857/items?limit=100&start=1000
83
https://api.zotero.org/groups/2808857/items?limit=100&start=1100
0


In [5]:
def download_file(download_url, filename):
    response = urllib.request.urlopen(download_url)    
    file = open(filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()

In [6]:
# now what? get the attachments
# inputs are jsonified version of your zotero group, base url to zotero group items, and api key
# going to have to loop through and run this for every page I think
def attachmentGrabber(rj, url, key):
    counter = 0
    to_extract = []
    citation_list = []
    for i in rj:
        item_key = rj[counter]["key"]
        headers = {"Zotero-API-Version":"3",'Connection':'close', "Zotero-API-Key":key}
        attach = requests.get(url + item_key + "/file", headers=headers)
        if "200" in str(attach):
            download_file(attach.url, f'Document_{item_key}')
            to_extract.append(f'Document_{item_key}.pdf')
            print("GOOD" + " 1 " + str(item_key))
        else:
            try:
                attach = requests.get(rj[counter]['links']['attachment']['href'] + "/file", headers=headers)
                print(attach)
                if "200" in str(attach):
                    download_file(attach.url, f'Document_{item_key}')
                    to_extract.append(f'Document_{item_key}.pdf')
                    print("GOOD" + " " + str(item_key))
                else:
                    print("that didn't work" + " " + attach.url)
            except Exception as e:
                print(e)
        counter +=1
    return to_extract

In [7]:
url = "https://api.zotero.org/groups/2808857/items/"
to_extract  = attachmentGrabber(rj, url, api_key)

<Response [200]>
GOOD DMKRZS8E
<Response [200]>
GOOD ZFPUQZNE
<Response [200]>
GOOD SYN9YTMS
<Response [200]>
GOOD G4923VSV
<Response [200]>
GOOD BTK3PPG8
<Response [200]>
GOOD EV273D9P
<Response [200]>
GOOD PDUA89RL
<Response [200]>
GOOD VZGACXNN
<Response [200]>
GOOD YGFJAA5F
<Response [200]>
GOOD KN8U5AXF
<Response [200]>
GOOD NDH25D7X
<Response [200]>
GOOD BM6L3JUC
<Response [200]>
GOOD TJR42XC5
<Response [200]>
GOOD RSB2STPR
<Response [200]>
GOOD 3JVVCGH6
<Response [200]>
GOOD 334IKY7D
<Response [200]>
GOOD NDTF8ZQG
<Response [200]>
GOOD UR68UWWC
<Response [200]>
GOOD P7ZQXA49
<Response [200]>
GOOD HE9X7NKT
<Response [200]>
GOOD 4S7CU23C
<Response [200]>
GOOD QKHD29TS
<Response [200]>
GOOD NYRUWCRI
<Response [200]>
GOOD 5H6TVKRX
<Response [200]>
GOOD WSFZDI5V
<Response [200]>
GOOD B32ETNP3
<Response [200]>
GOOD J5WD5TBV
<Response [200]>
GOOD ZH6G29C7
<Response [200]>
GOOD PGPVWJGT
<Response [200]>
GOOD 8MUZLDEB
<Response [200]>
GOOD EPBCSWG6
<Response [200]>
GOOD REYPV3ZC
<Respons

<Response [200]>
GOOD YI5SFAXG
<Response [200]>
GOOD WD89VNTJ
<Response [200]>
GOOD DFYLRQU9
<Response [200]>
GOOD 6NIL99HR
<Response [200]>
GOOD I4RZE3JR
<Response [200]>
GOOD 8HTICHII
GOOD 1 IKPJMQZG
GOOD 1 FJ9SMXH4
GOOD 1 82CHIVGP
<Response [200]>
GOOD LD9QKVIN
<Response [200]>
GOOD 5I39ZRDQ
<Response [200]>
GOOD PLBJZCVX
<Response [200]>
GOOD KEUSLH5T
<Response [200]>
GOOD QKT4INVU
<Response [200]>
GOOD 79XC3L7H
<Response [200]>
GOOD R5ALY5L9
<Response [200]>
GOOD FWYMMG2Q
<Response [200]>
GOOD T8KWFDLZ
<Response [200]>
GOOD EBHKBYNQ
<Response [200]>
GOOD 5M8VLMSI
<Response [200]>
GOOD FPKDJB68
<Response [200]>
GOOD 8CFBM2PQ
<Response [200]>
GOOD AZ78DWRJ
<Response [200]>
GOOD GDWN5XDJ
<Response [200]>
GOOD CMI4D6E8
<Response [200]>
GOOD 8SAMAF5H
<Response [200]>
GOOD GSK46ZTX
<Response [200]>
GOOD 9WKLYXHI
<Response [200]>
GOOD 8IM2RLBR
<Response [200]>
GOOD L48M4YU8
<Response [200]>
GOOD BY2IF7U8
<Response [200]>
GOOD BT3H6BXN
<Response [200]>
GOOD 2XQ489K3
<Response [200]>
GOOD 

GOOD 1 RU8BE4HI
GOOD 1 S6NWH553
GOOD 1 QBXA6NTI
GOOD 1 3G3JSU8V
GOOD 1 E3MGUHC4
GOOD 1 7CSKT7IK
GOOD 1 H5ZMFXST
GOOD 1 4JVNM7MT
GOOD 1 TB6ERCGW
GOOD 1 27663I6W
GOOD 1 W74AUWEF
GOOD 1 Z5U3DGWI
GOOD 1 H8Z87CAX
<Response [200]>
GOOD NQMFSY2S
GOOD 1 YR8T4JKL
GOOD 1 S673PU36
GOOD 1 LQDSDTM5
GOOD 1 LJEIWXST
GOOD 1 JEEPS3IF
GOOD 1 J4FVH3KZ
GOOD 1 DEST4TNH
GOOD 1 8MDBMXN5
GOOD 1 2K4Z4Q7H
GOOD 1 8LDTHSFW
GOOD 1 XAIQ6ULN
GOOD 1 RSQT98J3
GOOD 1 FRVCEZYV
GOOD 1 2RAEE8N4
GOOD 1 2HBPHZBH
GOOD 1 T7QPW9CJ
GOOD 1 T7S9NS4S
GOOD 1 9EFRFV79
GOOD 1 6IPWTKBS
GOOD 1 3J5UU3CJ
GOOD 1 WEPEYEIZ
GOOD 1 DAH2J9XC
GOOD 1 7KX8YQ28
GOOD 1 7AQN6654
GOOD 1 PRQU5M6T
GOOD 1 372DZPVY
GOOD 1 TRBJW7ZB
GOOD 1 U82AH78F
GOOD 1 R3QX5YAB
GOOD 1 LWP8CJEC
GOOD 1 8SKFXV3J
GOOD 1 RPJX4BKZ
GOOD 1 AD89KC8S
<Response [200]>
GOOD EVMFZZ2R
GOOD 1 ZUD6Q8L6
GOOD 1 MH4HZ3Q7
GOOD 1 WHH7UQ3Q
GOOD 1 AALHPEFA
GOOD 1 6KU8MFLE
'attachment'
GOOD 1 PC9K9YVG
'attachment'
GOOD 1 VBUYFCFV
GOOD 1 SMXKBVLB
GOOD 1 ZP6PSTJL
GOOD 1 NHGXI62N
GOOD 1 HPD7M8Q8


Before we extract the PDFs, this is where we should use the Scholarcy API to grab their references. https://ref.scholarcy.com/api/ We should play around with different return types to see what will be easiest. Part of me thinks it will be easiest to isolate references and compare them with the text in the PDF if we use the ```/extract``` endpoint to output in JSON.

In [8]:
scholarcy_headers = {"accept":"application/json","Authorization":"Bearer ", "Content-Type": 'multipart/form-data','Connection':'close' }
alt_headers = {"accept":"application/json","Authorization":"Bearer ", "Content-Type": 'multipart/form-data',"engine": "v2",'Connection':'close'}
url = "https://ref.scholarcy.com/api/references/extract"
refdict = {}
for i in to_extract:
    time.sleep(2)
    print("Working on " + i + "...")
    try:
        file = {'file': open(i, 'rb')}
        r = requests.post(url, files=file, data=scholarcy_headers)
        print(r)
        refdict[i] = r.json()
        print(i + " " + "is good to go")
    except Exception as e:
        print(str(e) + " " + "Trying API v2 instead...")
        try:
            r2 = requests.post(url, files=file,data=alt_headers)
            print(r2)
            refdict[i] = r2.json()
            print(i + " " + "is good to go")
        except Exception as e:
            print("That didn't work either...")


Working on Document_DMKRZS8E.pdf...
<Response [200]>
Document_DMKRZS8E.pdf is good to go
Working on Document_ZFPUQZNE.pdf...
<Response [200]>
Document_ZFPUQZNE.pdf is good to go
Working on Document_SYN9YTMS.pdf...
<Response [503]>
Expecting value: line 1 column 1 (char 0) Trying API v2 instead...
<Response [200]>
Document_SYN9YTMS.pdf is good to go
Working on Document_G4923VSV.pdf...
<Response [200]>
Document_G4923VSV.pdf is good to go
Working on Document_BTK3PPG8.pdf...
<Response [503]>
Expecting value: line 1 column 1 (char 0) Trying API v2 instead...
<Response [200]>
Document_BTK3PPG8.pdf is good to go
Working on Document_EV273D9P.pdf...
<Response [200]>
Document_EV273D9P.pdf is good to go
Working on Document_PDUA89RL.pdf...
<Response [200]>
Document_PDUA89RL.pdf is good to go
Working on Document_VZGACXNN.pdf...
<Response [200]>
Document_VZGACXNN.pdf is good to go
Working on Document_YGFJAA5F.pdf...
<Response [200]>
Document_YGFJAA5F.pdf is good to go
Working on Document_KN8U5AXF.pd

<Response [200]>
Document_Q3IV623L.pdf is good to go
Working on Document_9H6SKREV.pdf...
<Response [200]>
Document_9H6SKREV.pdf is good to go
Working on Document_QM36JDMB.pdf...
<Response [200]>
Document_QM36JDMB.pdf is good to go
Working on Document_82RY2G2B.pdf...
<Response [200]>
Document_82RY2G2B.pdf is good to go
Working on Document_LZYF75MM.pdf...
<Response [200]>
Document_LZYF75MM.pdf is good to go
Working on Document_BA7WW9VB.pdf...
<Response [200]>
Document_BA7WW9VB.pdf is good to go
Working on Document_G2JLFEP8.pdf...
<Response [200]>
Document_G2JLFEP8.pdf is good to go
Working on Document_E4BUALY4.pdf...
<Response [200]>
Document_E4BUALY4.pdf is good to go
Working on Document_D8B3UVXX.pdf...
<Response [200]>
Document_D8B3UVXX.pdf is good to go
Working on Document_P997PW8F.pdf...
<Response [200]>
Document_P997PW8F.pdf is good to go
Working on Document_5Q5NQKJW.pdf...
<Response [200]>
Document_5Q5NQKJW.pdf is good to go
Working on Document_PILB2N6R.pdf...
<Response [200]>
Docu

Working on Document_MK22CWCZ.pdf...
<Response [200]>
Document_MK22CWCZ.pdf is good to go
Working on Document_7UBGX3L4.pdf...
<Response [200]>
Document_7UBGX3L4.pdf is good to go
Working on Document_UQ94IYYS.pdf...
<Response [200]>
Document_UQ94IYYS.pdf is good to go
Working on Document_CZQHSS37.pdf...
<Response [200]>
Document_CZQHSS37.pdf is good to go
Working on Document_58SRB6U5.pdf...
<Response [200]>
Document_58SRB6U5.pdf is good to go
Working on Document_U8EIL2QK.pdf...
<Response [200]>
Document_U8EIL2QK.pdf is good to go
Working on Document_EVS9C3MJ.pdf...
<Response [200]>
Document_EVS9C3MJ.pdf is good to go
Working on Document_TJLGS45F.pdf...
<Response [503]>
Expecting value: line 1 column 1 (char 0) Trying API v2 instead...
<Response [200]>
Document_TJLGS45F.pdf is good to go
Working on Document_TF425XBV.pdf...
<Response [200]>
Document_TF425XBV.pdf is good to go
Working on Document_Z44DR29Y.pdf...
<Response [200]>
Document_Z44DR29Y.pdf is good to go
Working on Document_QYBIKX

<Response [200]>
Document_WD89VNTJ.pdf is good to go
Working on Document_DFYLRQU9.pdf...
<Response [200]>
Document_DFYLRQU9.pdf is good to go
Working on Document_6NIL99HR.pdf...
<Response [200]>
Document_6NIL99HR.pdf is good to go
Working on Document_I4RZE3JR.pdf...
<Response [200]>
Document_I4RZE3JR.pdf is good to go
Working on Document_8HTICHII.pdf...
<Response [200]>
Document_8HTICHII.pdf is good to go
Working on Document_IKPJMQZG.pdf...
<Response [200]>
Document_IKPJMQZG.pdf is good to go
Working on Document_FJ9SMXH4.pdf...
<Response [200]>
Document_FJ9SMXH4.pdf is good to go
Working on Document_82CHIVGP.pdf...
<Response [200]>
Document_82CHIVGP.pdf is good to go
Working on Document_LD9QKVIN.pdf...
<Response [200]>
Document_LD9QKVIN.pdf is good to go
Working on Document_5I39ZRDQ.pdf...
<Response [503]>
Expecting value: line 1 column 1 (char 0) Trying API v2 instead...
<Response [200]>
Document_5I39ZRDQ.pdf is good to go
Working on Document_PLBJZCVX.pdf...
<Response [200]>
Document_

Working on Document_6HSU4M2R.pdf...
<Response [503]>
Expecting value: line 1 column 1 (char 0) Trying API v2 instead...
<Response [200]>
Document_6HSU4M2R.pdf is good to go
Working on Document_N5INPBUV.pdf...
<Response [200]>
Document_N5INPBUV.pdf is good to go
Working on Document_TNJAM8SG.pdf...
<Response [200]>
Document_TNJAM8SG.pdf is good to go
Working on Document_F364P8AF.pdf...
<Response [200]>
Document_F364P8AF.pdf is good to go
Working on Document_JWHJU7WR.pdf...
<Response [200]>
Document_JWHJU7WR.pdf is good to go
Working on Document_MLNPAJKJ.pdf...
<Response [200]>
Document_MLNPAJKJ.pdf is good to go
Working on Document_NYUJZXC8.pdf...
<Response [200]>
Document_NYUJZXC8.pdf is good to go
Working on Document_CKGK4D4F.pdf...
<Response [200]>
Document_CKGK4D4F.pdf is good to go
Working on Document_8VR8R6ZT.pdf...
<Response [200]>
Document_8VR8R6ZT.pdf is good to go
Working on Document_PWQJJQ7B.pdf...
<Response [200]>
Document_PWQJJQ7B.pdf is good to go
Working on Document_AXMV9T

...

<Response [200]>
Document_ETZPPH7V.pdf is good to go
Working on Document_RU8BE4HI.pdf...
<Response [200]>
Document_RU8BE4HI.pdf is good to go
Working on Document_S6NWH553.pdf...
<Response [200]>
Document_S6NWH553.pdf is good to go
Working on Document_QBXA6NTI.pdf...
<Response [200]>
Document_QBXA6NTI.pdf is good to go
Working on Document_3G3JSU8V.pdf...
<Response [200]>
Document_3G3JSU8V.pdf is good to go
Working on Document_E3MGUHC4.pdf...
<Response [200]>
Document_E3MGUHC4.pdf is good to go
Working on Document_7CSKT7IK.pdf...
<Response [200]>
Document_7CSKT7IK.pdf is good to go
Working on Document_H5ZMFXST.pdf...
<Response [200]>
Document_H5ZMFXST.pdf is good to go
Working on Document_4JVNM7MT.pdf...
<Response [200]>
Document_4JVNM7MT.pdf is good to go
Working on Document_TB6ERCGW.pdf...
<Response [200]>
Document_TB6ERCGW.pdf is good to go
Working on Document_27663I6W.pdf...
<Response [200]>
Document_27663I6W.pdf is good to go
Working on Document_W74AUWEF.pdf...
<Response [200]>
Docu

Working on Document_LKM6ID8Q.pdf...
<Response [200]>
Document_LKM6ID8Q.pdf is good to go
Working on Document_WN7KM8SM.pdf...
<Response [200]>
Document_WN7KM8SM.pdf is good to go
Working on Document_AR2WEVGX.pdf...
<Response [200]>
Document_AR2WEVGX.pdf is good to go
Working on Document_CUJKW9BD.pdf...
<Response [503]>
Expecting value: line 1 column 1 (char 0) Trying API v2 instead...
<Response [200]>
Document_CUJKW9BD.pdf is good to go
Working on Document_632JKW5G.pdf...
<Response [200]>
Document_632JKW5G.pdf is good to go
Working on Document_8G94I8MW.pdf...
<Response [200]>
Document_8G94I8MW.pdf is good to go
Working on Document_59Z4XSFK.pdf...
<Response [200]>
Document_59Z4XSFK.pdf is good to go
Working on Document_5CM74MP2.pdf...
<Response [200]>
Document_5CM74MP2.pdf is good to go
Working on Document_5W6DJEBY.pdf...
<Response [200]>
Document_5W6DJEBY.pdf is good to go
Working on Document_2UJK5XN9.pdf...
<Response [200]>
Document_2UJK5XN9.pdf is good to go
Working on Document_QXGJ2X

<Response [200]>
Document_TM3PT4V5.pdf is good to go
Working on Document_23QPUKWQ.pdf...
<Response [200]>
Document_23QPUKWQ.pdf is good to go
Working on Document_2UYII2AE.pdf...
<Response [200]>
Document_2UYII2AE.pdf is good to go
Working on Document_TWC8JC27.pdf...
<Response [200]>
Document_TWC8JC27.pdf is good to go
Working on Document_89LKA9NP.pdf...
<Response [200]>
Document_89LKA9NP.pdf is good to go
Working on Document_W5UWLUPB.pdf...
<Response [200]>
Document_W5UWLUPB.pdf is good to go
Working on Document_4NVYFPE6.pdf...
<Response [200]>
Document_4NVYFPE6.pdf is good to go
Working on Document_VEPA7XZP.pdf...
<Response [200]>
Document_VEPA7XZP.pdf is good to go
Working on Document_UEPK37LX.pdf...
<Response [200]>
Document_UEPK37LX.pdf is good to go
Working on Document_9UA9V72C.pdf...
<Response [200]>
Document_9UA9V72C.pdf is good to go
Working on Document_8VWG55LK.pdf...
<Response [200]>
Document_8VWG55LK.pdf is good to go
Working on Document_IV4APKM8.pdf...
<Response [200]>
Docu

Working on Document_2QDWZVZA.pdf...
<Response [200]>
Document_2QDWZVZA.pdf is good to go
Working on Document_VVYAYDC2.pdf...
<Response [200]>
Document_VVYAYDC2.pdf is good to go
Working on Document_SFP9R7RY.pdf...
<Response [200]>
Document_SFP9R7RY.pdf is good to go
Working on Document_JS3LE6YU.pdf...
<Response [200]>
Document_JS3LE6YU.pdf is good to go
Working on Document_A7FC8QA4.pdf...
<Response [200]>
Document_A7FC8QA4.pdf is good to go
Working on Document_FS496FZR.pdf...
<Response [200]>
Document_FS496FZR.pdf is good to go
Working on Document_DQRHM6TU.pdf...
<Response [200]>
Document_DQRHM6TU.pdf is good to go
Working on Document_ZBZXESTX.pdf...
<Response [200]>
Document_ZBZXESTX.pdf is good to go
Working on Document_B2DRVURP.pdf...
<Response [200]>
Document_B2DRVURP.pdf is good to go
Working on Document_LXG2JY5U.pdf...
<Response [200]>
Document_LXG2JY5U.pdf is good to go
Working on Document_XBH6224E.pdf...
<Response [200]>
Document_XBH6224E.pdf is good to go
Working on Document_X

<Response [200]>
Document_YPCCYDEX.pdf is good to go
Working on Document_BI6W8NUW.pdf...
<Response [200]>
Document_BI6W8NUW.pdf is good to go
Working on Document_K282IYAT.pdf...
<Response [200]>
Document_K282IYAT.pdf is good to go
Working on Document_Z9FNYD7R.pdf...
<Response [200]>
Document_Z9FNYD7R.pdf is good to go
Working on Document_QXMXTLCQ.pdf...
<Response [200]>
Document_QXMXTLCQ.pdf is good to go
Working on Document_HJUJVZLY.pdf...
<Response [200]>
Document_HJUJVZLY.pdf is good to go
Working on Document_3LVZ63RU.pdf...
<Response [200]>
Document_3LVZ63RU.pdf is good to go


The attachments have been procured. Let's do something with them. Thanks to [PDF Text Miner](https://github.com/prldc/pdf_text_miner) for this function.

In [9]:
def extract_pdfs(list):  # You can easily extract a list from a .csv with pandas.
    d = {'file_name': ['dummy'], 'file_text': ['dummy'], 'ocr': [False]}
    df = pd.DataFrame(d, columns=['file_name', 'file_text', 'ocr'])
    count = 1
    for pdf in list:
        try:
            ext = os.path.splitext(pdf)[1][1:].strip()  # Gets file extension.
            if ext == 'pdf':  # Guarantees that the file is a .pdf, otherwise the program will crash when extracting text.
                ocr = False
                name = pdf.split('.pdf')[0]
                doc = fitz.open(f"{name}.pdf")
                text_file = open(f"{name}.txt", 'w')
                number_of_pages = doc.pageCount
                for page_n in range(number_of_pages):  # Extracts text from each page.
                    page = doc.load_page(page_n)
                    page_content = page.get_text("text")
                    text_file.write(page_content)
                if os.stat(
                        f"{name}.txt").st_size < 2000:  # Assumes file lacks OCR based on .txt file size, starts Tesseract.
                    ocr = True
                    os.remove(f"{name}.txt")  # Removes the previously scraped .txt.
                    tess_file = f"{name}.pdf"
                    pages = convert_from_path(tess_file, 500)
                    image_counter = 1
                    for page in pages:  # Converts the PDF to image.
                        filename = f"{name}page_{str(image_counter)}.jpg"
                        page.save(filename, 'JPEG')
                        image_counter = image_counter + 1
                    filelimit = image_counter - 1
                    outfile = f"{name}.txt"
                    f = open(outfile, "a")
                    for i in range(1, filelimit + 1):  # Applies OCR to each image, saves text file.
                        filename = f"{name}page_{str(i)}.jpg"
                        text = str((pytesseract.image_to_string(Image.open(filename), lang="por")))
                        text = text.replace('-\n', '')
                        f.write(text)
                    f.close()
                text = open(f"{name}.txt", 'r')
                txt = " ".join(text.readlines())
                df = df.append({'file_name': f"{name}", 'file_text': txt, 'ocr': ocr}, ignore_index=True)    
                end = datetime.datetime.now()
                print(
                    f"Finished {name} at {end}. OCR = {ocr}. {count} files read. {round(count * 100 / len(list), 2)}% done.")
        except:
            print(f'Did not finish {pdf}... check out that one.')
        count = count + 1

    return df

In [10]:
out = extract_pdfs(to_extract)

Finished Document_DMKRZS8E at 2022-08-22 14:20:41.592465. OCR = False. 1 files read. 0.09% done.
Finished Document_ZFPUQZNE at 2022-08-22 14:20:41.690634. OCR = False. 2 files read. 0.19% done.
Finished Document_SYN9YTMS at 2022-08-22 14:20:41.851428. OCR = False. 3 files read. 0.28% done.
Finished Document_G4923VSV at 2022-08-22 14:20:41.894435. OCR = False. 4 files read. 0.37% done.
Finished Document_BTK3PPG8 at 2022-08-22 14:20:42.227425. OCR = False. 5 files read. 0.47% done.
Finished Document_EV273D9P at 2022-08-22 14:20:42.268104. OCR = False. 6 files read. 0.56% done.
Finished Document_PDUA89RL at 2022-08-22 14:20:42.298090. OCR = False. 7 files read. 0.65% done.
Finished Document_VZGACXNN at 2022-08-22 14:20:42.410594. OCR = False. 8 files read. 0.75% done.
Finished Document_YGFJAA5F at 2022-08-22 14:20:42.536113. OCR = False. 9 files read. 0.84% done.
Finished Document_KN8U5AXF at 2022-08-22 14:20:42.579671. OCR = False. 10 files read. 0.93% done.
Finished Document_NDH25D7X at

Finished Document_QNFQZBRW at 2022-08-22 14:22:21.448836. OCR = False. 85 files read. 7.93% done.
Finished Document_5ZDP8VN5 at 2022-08-22 14:22:21.484759. OCR = False. 86 files read. 8.02% done.
Finished Document_4EV2KSBG at 2022-08-22 14:22:21.597025. OCR = False. 87 files read. 8.12% done.
Finished Document_BC9JSIBR at 2022-08-22 14:22:21.668653. OCR = False. 88 files read. 8.21% done.
Finished Document_EZ74L3J8 at 2022-08-22 14:22:21.739941. OCR = False. 89 files read. 8.3% done.
Finished Document_Q3IV623L at 2022-08-22 14:22:21.771110. OCR = False. 90 files read. 8.4% done.
Finished Document_9H6SKREV at 2022-08-22 14:22:21.866608. OCR = False. 91 files read. 8.49% done.
Finished Document_QM36JDMB at 2022-08-22 14:22:22.026312. OCR = False. 92 files read. 8.58% done.
Finished Document_82RY2G2B at 2022-08-22 14:22:22.099120. OCR = False. 93 files read. 8.68% done.
Finished Document_LZYF75MM at 2022-08-22 14:22:22.145018. OCR = False. 94 files read. 8.77% done.
Finished Document_BA7W

Finished Document_TKLJFV2A at 2022-08-22 14:26:50.450990. OCR = False. 168 files read. 15.67% done.
Finished Document_XUZMUJUK at 2022-08-22 14:26:51.042577. OCR = False. 169 files read. 15.76% done.
Finished Document_4UV8IP5P at 2022-08-22 14:26:51.218123. OCR = False. 170 files read. 15.86% done.
Finished Document_MLBV8HU8 at 2022-08-22 14:26:51.301768. OCR = False. 171 files read. 15.95% done.
Finished Document_38KMLSRB at 2022-08-22 14:26:51.402444. OCR = False. 172 files read. 16.04% done.
Finished Document_BALJEPTU at 2022-08-22 14:26:51.506228. OCR = False. 173 files read. 16.14% done.
Finished Document_EPUKEEM8 at 2022-08-22 14:26:51.537905. OCR = False. 174 files read. 16.23% done.
Finished Document_ESHYVFU9 at 2022-08-22 14:26:51.574081. OCR = False. 175 files read. 16.32% done.
Finished Document_IDHZ8WI5 at 2022-08-22 14:26:51.610401. OCR = False. 176 files read. 16.42% done.
Finished Document_QKZB9R8N at 2022-08-22 14:26:51.662953. OCR = False. 177 files read. 16.51% done.


Finished Document_GCMG5P5J at 2022-08-22 14:28:38.406682. OCR = False. 254 files read. 23.69% done.
Finished Document_8MA5JWD5 at 2022-08-22 14:28:38.679576. OCR = False. 255 files read. 23.79% done.
Finished Document_5FN7IIPD at 2022-08-22 14:28:38.779435. OCR = False. 256 files read. 23.88% done.
Finished Document_V4AP8WFX at 2022-08-22 14:28:38.835907. OCR = False. 257 files read. 23.97% done.
Finished Document_EZL64X72 at 2022-08-22 14:28:38.900252. OCR = False. 258 files read. 24.07% done.
Finished Document_JLHV9QB5 at 2022-08-22 14:28:38.952330. OCR = False. 259 files read. 24.16% done.
Finished Document_JI4B2NX5 at 2022-08-22 14:28:38.979556. OCR = False. 260 files read. 24.25% done.
Finished Document_X6BF2L98 at 2022-08-22 14:28:39.101138. OCR = False. 261 files read. 24.35% done.
Finished Document_QINZVK9G at 2022-08-22 14:28:39.181779. OCR = False. 262 files read. 24.44% done.
Finished Document_BBJ8XCU2 at 2022-08-22 14:28:39.241156. OCR = False. 263 files read. 24.53% done.


Finished Document_EDECKS3H at 2022-08-22 14:28:44.958816. OCR = False. 339 files read. 31.62% done.
Finished Document_KQISWICM at 2022-08-22 14:28:45.008255. OCR = False. 340 files read. 31.72% done.
Finished Document_K557V25A at 2022-08-22 14:28:45.098628. OCR = False. 341 files read. 31.81% done.
Finished Document_2ZUFT3FH at 2022-08-22 14:28:45.140007. OCR = False. 342 files read. 31.9% done.
Finished Document_F6IT8R9P at 2022-08-22 14:28:45.229124. OCR = False. 343 files read. 32.0% done.
Finished Document_GV5ISQKI at 2022-08-22 14:28:45.307517. OCR = False. 344 files read. 32.09% done.
Finished Document_G2T8C834 at 2022-08-22 14:28:45.351768. OCR = False. 345 files read. 32.18% done.
Finished Document_4BV85SKQ at 2022-08-22 14:28:45.403696. OCR = False. 346 files read. 32.28% done.
Finished Document_SJ4KJGTV at 2022-08-22 14:28:45.451682. OCR = False. 347 files read. 32.37% done.
Finished Document_B9828Z6A at 2022-08-22 14:28:45.543486. OCR = False. 348 files read. 32.46% done.
Fi

...

Finished Document_Q8W2ZDB8 at 2022-08-22 14:55:13.031251. OCR = False. 679 files read. 63.34% done.
Finished Document_8G3QRWM6 at 2022-08-22 14:55:13.079377. OCR = False. 680 files read. 63.43% done.
Finished Document_AH9EZQIT at 2022-08-22 14:55:13.185158. OCR = False. 681 files read. 63.53% done.
Finished Document_2U3G6BTX at 2022-08-22 14:55:13.331052. OCR = False. 682 files read. 63.62% done.
Finished Document_T5NDXNMK at 2022-08-22 14:55:13.381257. OCR = False. 683 files read. 63.71% done.
Finished Document_S3XQWWCW at 2022-08-22 14:55:13.457937. OCR = False. 684 files read. 63.81% done.
Finished Document_GIJ6BH8W at 2022-08-22 14:55:13.492428. OCR = False. 685 files read. 63.9% done.
Finished Document_4222JNBR at 2022-08-22 14:55:13.677599. OCR = False. 686 files read. 63.99% done.
Finished Document_4C2XTZGI at 2022-08-22 14:55:13.698554. OCR = False. 687 files read. 64.09% done.
Finished Document_KV5I5TWM at 2022-08-22 14:55:13.750222. OCR = False. 688 files read. 64.18% done.
D

Did not finish Document_NHGXI62N.pdf... check out that one.
Finished Document_HPD7M8Q8 at 2022-08-22 14:59:15.909056. OCR = False. 768 files read. 71.64% done.
Finished Document_VNI4UXDR at 2022-08-22 14:59:15.979370. OCR = False. 769 files read. 71.74% done.
Did not finish Document_V3HL89PY.pdf... check out that one.
Finished Document_6KYHB7KB at 2022-08-22 14:59:16.030082. OCR = False. 771 files read. 71.92% done.
Did not finish Document_6NBX33WR.pdf... check out that one.
Did not finish Document_DBGAI4EI.pdf... check out that one.
Finished Document_VPTYM2ET at 2022-08-22 14:59:16.069513. OCR = False. 774 files read. 72.2% done.
Finished Document_3VPK56UW at 2022-08-22 14:59:16.090518. OCR = False. 775 files read. 72.29% done.
Finished Document_BX2ZVG2H at 2022-08-22 14:59:16.169120. OCR = False. 776 files read. 72.39% done.
Finished Document_XTBLA5TJ at 2022-08-22 14:59:16.225215. OCR = False. 777 files read. 72.48% done.
Did not finish Document_AC82SKJ5.pdf... check out that one.
F

Finished Document_R4NBCXC6 at 2022-08-22 14:59:25.578582. OCR = False. 859 files read. 80.13% done.
Did not finish Document_PLX3YFQJ.pdf... check out that one.
Finished Document_MQWRIE7L at 2022-08-22 14:59:25.669217. OCR = False. 861 files read. 80.32% done.
Finished Document_62W22LDB at 2022-08-22 14:59:25.705705. OCR = False. 862 files read. 80.41% done.
Finished Document_MW9HBF22 at 2022-08-22 14:59:25.753737. OCR = False. 863 files read. 80.5% done.
Finished Document_MFA9NVRD at 2022-08-22 14:59:25.788753. OCR = False. 864 files read. 80.6% done.
Finished Document_4AXG9ZTM at 2022-08-22 14:59:25.842757. OCR = False. 865 files read. 80.69% done.
Finished Document_D5UFIUJA at 2022-08-22 14:59:25.870105. OCR = False. 866 files read. 80.78% done.
Finished Document_N6ZPQ8FP at 2022-08-22 14:59:25.915789. OCR = False. 867 files read. 80.88% done.
Finished Document_VC2N9ZKH at 2022-08-22 14:59:25.975151. OCR = False. 868 files read. 80.97% done.
Finished Document_IUIWB47P at 2022-08-22 1

Finished Document_3WNRQ9VG at 2022-08-22 15:06:33.435905. OCR = False. 944 files read. 88.06% done.
Finished Document_THZL4GEX at 2022-08-22 15:06:33.513873. OCR = False. 945 files read. 88.15% done.
Finished Document_WV9DTCAW at 2022-08-22 15:06:33.544849. OCR = False. 946 files read. 88.25% done.
Finished Document_B3YE875Y at 2022-08-22 15:06:33.624329. OCR = False. 947 files read. 88.34% done.
Finished Document_CR3FE8EN at 2022-08-22 15:06:33.666267. OCR = False. 948 files read. 88.43% done.
Finished Document_KK5G7FMQ at 2022-08-22 15:06:33.726988. OCR = False. 949 files read. 88.53% done.
Finished Document_86I56GVH at 2022-08-22 15:06:33.840558. OCR = False. 950 files read. 88.62% done.
Finished Document_3C5SXZBJ at 2022-08-22 15:06:33.919618. OCR = False. 951 files read. 88.71% done.
Finished Document_MILR9STY at 2022-08-22 15:06:33.978998. OCR = False. 952 files read. 88.81% done.
Finished Document_8SGR4JNH at 2022-08-22 15:06:34.054912. OCR = False. 953 files read. 88.9% done.
F

Finished Document_748NTPRA at 2022-08-22 15:10:26.251085. OCR = False. 1033 files read. 96.36% done.
Finished Document_HI9S5YKN at 2022-08-22 15:10:26.357371. OCR = False. 1034 files read. 96.46% done.
Finished Document_C6INMS2R at 2022-08-22 15:10:26.402365. OCR = False. 1035 files read. 96.55% done.
Finished Document_693FKJSV at 2022-08-22 15:10:26.473935. OCR = False. 1036 files read. 96.64% done.
Finished Document_87S8CV5Y at 2022-08-22 15:10:26.542895. OCR = False. 1037 files read. 96.74% done.
Finished Document_6VV7U8ZR at 2022-08-22 15:13:21.485070. OCR = True. 1038 files read. 96.83% done.
Finished Document_ABSGQGFV at 2022-08-22 15:13:21.532756. OCR = False. 1039 files read. 96.92% done.
Finished Document_5HNXDXWB at 2022-08-22 15:13:22.048355. OCR = False. 1040 files read. 97.01% done.
Finished Document_5VGQCWNM at 2022-08-22 15:13:22.077510. OCR = False. 1041 files read. 97.11% done.
Finished Document_2GA9XM3Y at 2022-08-22 15:13:22.161081. OCR = False. 1042 files read. 97.2

In [11]:
out # we are able to scrape the vast majority of the articles without a problem

,file_name,file_text,ocr
0,dummy,dummy,False
1,Document_DMKRZS8E,\n \n \n \n Electronic Delivery Cover Shee...,False
2,Document_ZFPUQZNE,Will You Accept an Imperfect AI? Exploring Des...,False
3,Document_SYN9YTMS,When (ish) is My Bus? User-centered Visualizat...,False
4,Document_G4923VSV,What are Data Insights to Professional Visuali...,False
...,...,...,...
1010,Document_K282IYAT,\n \n A future for cities? Building new meth...,False
1011,Document_Z9FNYD7R,Article\n Urban Analytics and\n City Science\n...,False
1012,Document_QXMXTLCQ,https://doi.org/10.1177/0739456X18793716\n Jou...,False
1013,Document_HJUJVZLY,Editorial\n Urban Analytics and\n City Science...,False


In [14]:
#for rapid topic modeling/cleaning prototyping:
out.to_csv("extracted_text.csv")
with open('refs.txt', 'w') as convert_file:
     convert_file.write(json.dumps(refdict))

### #2. Cleaning up the text
Using strategies based on [this article](https://monkeylearn.com/blog/text-cleaning/).

Here, before we do any other cleaning tasks, this is where we need to match and remove references with the information we got from the Scholarcy API above.

#### Resources:
- [String comparison in Python](https://note.nkmk.me/en/python-str-compare/)
- [Potentially useful example on StackOverflow](https://stackoverflow.com/questions/39551029/if-else-statement-for-finding-the-index-of-a-character-in-a-string)

In [26]:
def referenceChecker(extracted_df):
    ref_excluded = {}
    counter = 0
    string3=""
    matches=0
    nomatch=0
    for name in extracted_df['file_name']:
        if "dummy" not in name:
            text_file = open(str(name) +".txt", "r")
            data = text_file.read()      # Read whole file to a string
            text_file.close()         # Close file
            string1 = data.replace('\n',"")
            string2 = string1.replace('\t',"")
            for i in refdict[str(name) + ".pdf"]["references"]:
                if i[0:24] in string2:
                    starti =string2.find(i[0:24])
                    endi = int(starti) + len(i)
                    matches+=1
                    string3 = string2.replace(string2[starti:endi], "")
                else:
                    pass
                    nomatch +=1
            ref_excluded[name] = string3
    print("Matches: " + str(matches) + " No Matches: " +str(nomatch) )
    return ref_excluded

In [29]:
ref_ex = referenceChecker(out) #needs some work

Matches: 37669 No Matches: 7777


### Cleaning Tasks
- Case Normalization
- Remove Unicode Characters
    - In the future, we may want to experiment with using [ftfy](https://github.com/rspeer/python-ftfy), which fixes text encoding issues, in this pipeline. We may also be interested in exploring [scrubadub](https://scrubadub.readthedocs.io/en/stable/index.html), which redacts potential PII from text.
- Remove Stopwords
- Lemmatize

In [30]:
def textCleaner(ref_ex_dict):
    full_corpus = []
    for i in ref_ex_dict.keys():
        data = ref_ex_dict[i]
        da = data.lower()
        d = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", da)
        stop = stopwords.words('english')
        all_words = nltk.word_tokenize(d)
        words = [w for w in all_words if w not in stop]
        words = [w for w in words if w.isalpha()]
        from nltk.stem import WordNetLemmatizer
        lemmatizer = WordNetLemmatizer()
        word_out = []
        for word in words:
            a = lemmatizer.lemmatize(word)
            word_out.append(a)
        ref_ex_dict[i] = word_out
        full_corpus.append(word_out)
    return ref_ex_dict, full_corpus

In [31]:
a = textCleaner(ref_ex)

In [32]:
corpusdict = a[0]
corpus = a[1]
full_corpus = [x for xs in corpus for x in xs] #flattened

## Basic analysis

#### A quick frequency distribution of the *most common words* in the corpus, and the *most common two and three word collocations* in the corpus.

In [33]:
word_fd = nltk.FreqDist(full_corpus)
word_fd

FreqDist({'system': 58124, 'data': 57900, 'urban': 49515, 'model': 43761, 'city': 41599, 'planning': 36355, 'design': 34819, 'information': 26183, 'process': 24810, 'et': 24106, ...})

In [34]:
bigram_fd = nltk.FreqDist(nltk.bigrams(full_corpus))
bigram_fd

FreqDist({('et', 'al'): 20250, ('smart', 'city'): 8670, ('expert', 'system'): 6067, ('urban', 'planning'): 5958, ('big', 'data'): 5761, ('urban', 'design'): 4503, ('artificial', 'intelligence'): 4451, ('machine', 'learning'): 4440, ('land', 'use'): 4312, ('neural', 'network'): 3876, ...})

In [35]:
trigram_fd = nltk.FreqDist(nltk.trigrams(full_corpus))
trigram_fd

FreqDist({('decision', 'support', 'system'): 2035, ('planning', 'support', 'system'): 940, ('geographic', 'information', 'system'): 838, ('b', 'b', 'b'): 682, ('big', 'data', 'analytics'): 632, ('environment', 'planning', 'b'): 556, ('urban', 'information', 'system'): 549, ('new', 'york', 'ny'): 528, ('proceeding', 'international', 'conference'): 501, ('intelligent', 'transportation', 'system'): 467, ...})

#### Using some of nltk's built in functions to get more information about the collocation scores according to association measures.
See more information about the nltk collocation methodology [here](https://www.nltk.org/api/nltk.collocations.html)

In [36]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder(word_fd, bigram_fd)
finder.score_ngrams(bigram_measures.raw_freq)

[(('et', 'al'), 0.0029725154614841263),
 (('smart', 'city'), 0.0012726769901761667),
 (('expert', 'system'), 0.000890580311349343),
 (('urban', 'planning'), 0.000874580104667774),
 (('big', 'data'), 0.0008456622999313606),
 (('urban', 'design'), 0.0006609993641018776),
 (('artificial', 'intelligence'), 0.0006533662379785603),
 (('machine', 'learning'), 0.0006517515382217047),
 (('land', 'use'), 0.0006329623046873853),
 (('neural', 'network'), 0.0005689614779611098),
 (('support', 'system'), 0.000538135391693867),
 (('decision', 'support'), 0.0005014376699471494),
 (('new', 'york'), 0.0004999697610772807),
 (('information', 'system'), 0.0004880796992313442),
 (('international', 'conference'), 0.0003352703858780121),
 (('decision', 'making'), 0.000333068522573209),
 (('case', 'study'), 0.00031897659742246946),
 (('articial', 'intelligence'), 0.000297985500583347),
 (('design', 'process'), 0.00026730620520309104),
 (('planning', 'process'), 0.00024367287239820493),
 (('transportation', 'p

### Optional File Cleanup

In [ ]:
for f in glob.glob("Document_*"):
    os.remove(f)